# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [1]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [2]:
# Use this cell for any import statements you add
import shutil


---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [3]:
artists = {'alan':"https://www.azlyrics.com/j/jacksonal.html",
           'queen':"https://www.azlyrics.com/q/queen.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

***A: Scraping the artist pages (e.g., https://www.azlyrics.com/q/queen.html and https://www.azlyrics.com/j/jacksonal.html) is allowed according to the `robots.txt` file (https://www.azlyrics.com/robots.txt) because these pages are not in the disallowed directories `/lyricsdb/` or `/song/`. The individual lyrics pages contain `/lyrics/` in the URL, in the following format (e.g., `https://---/lyrics/jacksonal/---.html`), which is not explicitly disallowed in the `robots.txt` file.***

### Compile all of the URLs for the song lyrics available for each artist identified above


In [4]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # Parse the HTML content of the page
    soup = BeautifulSoup(r.content, 'html.parser')

    # Find all anchor tags with href attributes in the parsed HTML
    for link in soup.find_all('a', href=True):
        url = link['href'] 
    # Append full lyrics URL link (value) to each Artist (key) in the dictionary
        if url.startswith("/lyrics/"): # Returns boolean value
            full_url = "https://www.azlyrics.com" + url
            lyrics_pages[artist].append(full_url)

# Print the 'value' (link) for each key (Artist) in the dictionary of lists  
for artist, links in lyrics_pages.items():
    print(f"Lyrics pages for {artist}:")
    for link in links:
        print(link)

Lyrics pages for alan:
https://www.azlyrics.com/lyrics/alanjackson/aintyourmemorygotnoprideatall.html
https://www.azlyrics.com/lyrics/alanjackson/justforgetitson.html
https://www.azlyrics.com/lyrics/alanjackson/merleandgeorge.html
https://www.azlyrics.com/lyrics/alanjackson/donttouchme.html
https://www.azlyrics.com/lyrics/alanjackson/breakoutthegoodstuff.html
https://www.azlyrics.com/lyrics/alanjackson/thestealofthenight.html
https://www.azlyrics.com/lyrics/alanjackson/whenthecatgoesout.html
https://www.azlyrics.com/lyrics/alanjackson/icouldntcaremore.html
https://www.azlyrics.com/lyrics/alanjackson/theycallmeaplayboy.html
https://www.azlyrics.com/lyrics/alanjackson/wleeodanielandthelightcrustdoughboys.html
https://www.azlyrics.com/lyrics/alanjackson/yourenotdrinkingenough.html
https://www.azlyrics.com/lyrics/alanjackson/aceofhearts.html
https://www.azlyrics.com/lyrics/alanjackson/hereintherealworld.html
https://www.azlyrics.com/lyrics/alanjackson/bluebloodedwoman.html
https://www.azly

Let's make sure we have enough lyrics pages to scrape. 

In [5]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [6]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For alan we have 280.
The full pull will take for this artist will take 0.78 hours.
For queen we have 197.
The full pull will take for this artist will take 0.55 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [7]:
def generate_filename_from_url(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [8]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

### Inspect page for structure

In [9]:
# Page URL
url = "https://www.azlyrics.com/lyrics/queen/thefairyfellersmasterstroke.html"

# Request the page
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all <div> elements on the page
divs = soup.find_all('div')

# Count the number of <div> elements
div_count = len(divs)

print(f"Total number of <div> elements: {div_count}")

# Print the content of each <div> with its index to verify
for index, div in enumerate(divs):
    print(f"Index {index}: {div.get_text(strip=True)[:100]}")

### Scrape Lyric from Site and write to text files seprated by 'Artist' folders

In [10]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist, pages in lyrics_pages.items():

    # 1. Build a subfolder for the artist
    artist_folder = os.path.join("lyrics", artist)
    if not os.path.exists(artist_folder):
        os.makedirs(artist_folder)
        
    # 2. Iterate over the lyrics pages
    for i, lyrics_page_url in enumerate(pages[:50]): # 50 songs for each artist
        try:
            # 3. Request the lyrics page. 
            time.sleep(5 + 10 * random.random())
            response = requests.get(lyrics_page_url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # 4. Extract the title and lyrics from the page (div 24 based on above inspection)
                title = soup.find('title').get_text(strip=True)
                lyrics_div = soup.find_all('div')[24]
                lyrics = lyrics_div.get_text(separator="\n", strip=True)

                
                # 5. Use `generate_filename_from_url` to generate the filename. 
                filename = generate_filename_from_url(lyrics_page_url)  
                file_path = os.path.join(artist_folder, filename) 
                # Write out the title, two returns ('\n'), and the lyrics (in the text file in the folder)
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(title + '\n\n') 
                    f.write(lyrics)
                                
                total_pages += 1 # update counter
                print(f'Successfully wrote lyrics for {title} to {file_path}')
            else: # Check if error is on our side or not 
                print(f'Failed to retrieve page: {lyrics_page_url}, Status code: {response.status_code}') 
                        
        except Exception as e: 
            print(f'An error occurred while processing {lyrics_page_url}: {e}')

end = time.time()

Successfully wrote lyrics for Alan Jackson - Ain't Your Memory Got No Pride At All Lyrics | AZLyrics.com to lyrics\alan\httpswww_azlyrics_comalanjackson_aintyourmemorygotnoprideatall.txt
Successfully wrote lyrics for Alan Jackson - Just Forget It, Son Lyrics | AZLyrics.com to lyrics\alan\httpswww_azlyrics_comalanjackson_justforgetitson.txt
Successfully wrote lyrics for Alan Jackson - Merle And George Lyrics | AZLyrics.com to lyrics\alan\httpswww_azlyrics_comalanjackson_merleandgeorge.txt
Successfully wrote lyrics for Alan Jackson - Don't Touch Me Lyrics | AZLyrics.com to lyrics\alan\httpswww_azlyrics_comalanjackson_donttouchme.txt
Successfully wrote lyrics for Alan Jackson - Break Out The Good Stuff Lyrics | AZLyrics.com to lyrics\alan\httpswww_azlyrics_comalanjackson_breakoutthegoodstuff.txt
Successfully wrote lyrics for Alan Jackson - The Steal Of The Night Lyrics | AZLyrics.com to lyrics\alan\httpswww_azlyrics_comalanjackson_thestealofthenight.txt
Successfully wrote lyrics for Alan 

In [11]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.3 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [12]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [13]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For alan we have 50 files.
For alan we have roughly 10971 words, 1319 are unique.
For queen we have 50 files.
For queen we have roughly 11885 words, 1698 are unique.


<center><b>References:</b></center>


- Weber, B. (2023, August 1). Python enumerate(): *Simplify loops that need counters.* Real Python. https://realpython.com/python-enumerate/
- Sulcas, A. (2023, June 6). *Beautiful soup tutorial - how to Parse web data with python.* Oxylabs. https://oxylabs.io/blog/beautiful-soup-parsing-tutorial 
- ScrapFly Blog. (2024, August 22). *How to Parse Web Data with Python and Beautifulsoup.* ScrapFly Blog. https://scrapfly.io/blog/web-scraping-with-python-beautifulsoup/
- Sanchhaya Education Private Limited. (2024, July 7). *Python | os.path.join() method.* GeeksforGeeks. https://www.geeksforgeeks.org/python-os-path-join-method/ 
- Python Software Foundation. (n.d.). *os.path — Common pathname manipulations.* Python documentation. https://docs.python.org/3/library/os.path.html
- Python Software Foundation. (n.d.). *Built-in functions - Open.* Python documentation. https://docs.python.org/3/library/functions.html#open
- Python Software Foundation. (n.d.). *Built-in functions - Enumerate.* Python documentation. https://docs.python.org/3/library/functions.html#enumerate
- OpenAI. (2023). ChatGPT (September 9 version) [Large language model]. https://chat.openai.com/
- Mozilla Developer Network. (n.d.). *String.prototype.startsWith().* MDN Web Docs. https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/String/startsWith
- Fawole, J. (2024, May 27). *How to Scrape Web Data Using Requests and BeautifulSoup.* ScraperAPI. https://www.scraperapi.com/blog/beautifulsoup-and-requests/
- Awan, A. A. (2023, February 24). *Exception & Error Handling in python: Tutorial by datacamp.* DataCamp. https://www.datacamp.com/tutorial/exception-handling-python
- Albrecht, J., Ramachandran, S., & Winkler, C. (2020). *Blueprints for text analytics using Python.* O'Reilly.
- *Python File write() Method.* W3Schools Online Web Tutorials. (n.d.). https://www.w3schools.com/python/ref_file_write.asp
